In [1]:
from google.colab import drive
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np
%matplotlib inline
configure_plotly_browser_state()
ROOT = '/content/drive'     # Raíz del Drive por Default
PROJ = '/content/drive/My Drive/covid'       # Dirección del proyecto en donde este

drive.mount(ROOT, force_remount=True)           # Montamos el drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
df_train = pd.read_csv(PROJ+'/data/train.csv')
df_test = pd.read_csv(PROJ+'/data/train.csv')
df_submissions = pd.read_csv(PROJ+'/data/submission.csv')

In [3]:
df_train.head(5)

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In [4]:
df_test.head(5)

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0,65.0,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0,65.0,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0,65.0,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0,65.0,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0,65.0,2020-01-26,0.0,0.0


In [0]:
#Convert to date time to better later manipulation
df_train['Date'] = pd.to_datetime(df_train.Date)
df_test['Date'] = pd.to_datetime(df_test.Date)

In [0]:
def missing_values(df):
    s_types = df.dtypes
    s_missing= df.isnull().sum()
    s_missingper= (df.isnull().sum()/df.shape[0])*100

    explo = pd.DataFrame({'Types': s_types,
                       'Missing': s_missing,
                      'Missing%': s_missingper,}).sort_values(by=['Missing%','Types'],ascending=False)
    return explo

def description(df):
    s_types = df.dtypes
    s_head= df.apply(lambda x: x[0:3].tolist())

    explo = pd.DataFrame({'Types': s_types,
                      'Head': s_head}).sort_values(by=['Types'],ascending=False)
    return explo

In [7]:
#Missing Values Train
missing_values(df_train)

,Types,Missing,Missing%
Province/State,object,9086,54.225352
Lat,float64,59,0.352113
Long,float64,59,0.352113
Country/Region,object,0,0.000000
Date,datetime64[ns],0,0.000000
ConfirmedCases,float64,0,0.000000
Fatalities,float64,0,0.000000
Id,int64,0,0.000000


In [8]:
#Missing Values Test
missing_values(df_test)

,Types,Missing,Missing%
Province/State,object,9086,54.225352
Lat,float64,59,0.352113
Long,float64,59,0.352113
Country/Region,object,0,0.000000
Date,datetime64[ns],0,0.000000
ConfirmedCases,float64,0,0.000000
Fatalities,float64,0,0.000000
Id,int64,0,0.000000


In [9]:
#Description
description(df_train)

,Types,Head
Province/State,object,"[nan, nan, nan]"
Country/Region,object,"[Afghanistan, Afghanistan, Afghanistan]"
Lat,float64,"[33.0, 33.0, 33.0]"
Long,float64,"[65.0, 65.0, 65.0]"
Id,int64,"[1, 2, 3]"
Date,datetime64[ns],"[2020-01-22 00:00:00, 2020-01-23 00:00:00, 202..."
ConfirmedCases,float64,"[0.0, 0.0, 0.0]"
Fatalities,float64,"[0.0, 0.0, 0.0]"


In [10]:
#Description
description(df_test)

,Types,Head
Province/State,object,"[nan, nan, nan]"
Country/Region,object,"[Afghanistan, Afghanistan, Afghanistan]"
Lat,float64,"[33.0, 33.0, 33.0]"
Long,float64,"[65.0, 65.0, 65.0]"
Id,int64,"[1, 2, 3]"
Date,datetime64[ns],"[2020-01-22 00:00:00, 2020-01-23 00:00:00, 202..."
ConfirmedCases,float64,"[0.0, 0.0, 0.0]"
Fatalities,float64,"[0.0, 0.0, 0.0]"


In [0]:
from pandas_profiling import ProfileReport

In [0]:
import seaborn as sns 
import matplotlib.pyplot as plt
sns.set()
sns.pairplot(df_train)
plt.show()

In [0]:
train_agg= df_train[['Country/Region','Date','ConfirmedCases','Fatalities']].groupby(['Country/Region','Date'],as_index=False).agg({'ConfirmedCases': 'sum', 'Fatalities': 'sum'})



In [18]:
import IPython
IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode, plot
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

fig = px.line(train_agg, x='Date', y='ConfirmedCases', color="Country/Region", hover_name="Country/Region")
fig.update_layout(autosize=False,width=1000,height=500,title='Casos Confirmados por tiempo en cada país')
fig.show()